In [2]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import glob
import re

In [3]:
all_historical = glob.glob("*-cabi-trip-history-data.csv")

In [89]:
historical_df_list = []
for each_file in all_historical:
    print each_file
    temp_df = pd.read_csv(each_file)
    temp_df.columns = [each.lower() for each in temp_df.columns]
    if 'start station number' in temp_df.columns:
        # This is a updated version of dataset
        temp_df = temp_df[['start date','end date','start station','start station number']]
        historical_df_list.append(temp_df)
    else:
        # older ones 
        temp_df = temp_df[['start date','end date','start station']]
        historical_df_list.append(temp_df)



2010-Q4-cabi-trip-history-data.csv
2011-Q1-cabi-trip-history-data.csv
2011-Q2-cabi-trip-history-data.csv
2011-Q3-cabi-trip-history-data.csv
2011-Q4-cabi-trip-history-data.csv
2012-Q1-cabi-trip-history-data.csv
2012-Q2-cabi-trip-history-data.csv
2012-Q3-cabi-trip-history-data.csv
2012-Q4-cabi-trip-history-data.csv
2013-Q1-cabi-trip-history-data.csv
2013-Q2-cabi-trip-history-data.csv
2013-Q3-cabi-trip-history-data.csv
2013-Q4-cabi-trip-history-data.csv
2014-Q1-cabi-trip-history-data.csv
2014-Q2-cabi-trip-history-data.csv
2014-Q3-cabi-trip-history-data.csv
2014-Q4-cabi-trip-history-data.csv
2015-Q1-cabi-trip-history-data.csv
2015-Q2-cabi-trip-history-data.csv
2015-Q3-cabi-trip-history-data.csv
2015-Q4-cabi-trip-history-data.csv
2016-Q1-cabi-trip-history-data.csv
2016-Q2-cabi-trip-history-data.csv


In [90]:
station_change = []
i = 0
for each_df in historical_df_list:
    print i
    # Filter out the changed names
    extract_start = each_df['start station'].str.extract('(?P<extracted>[^\[\]]+) *(?P<change>\[[^\[\]]+\])? *', expand=False)
#     extract_end = each_df['end station'].str.extract('(?P<extracted>[^\[\]]+) *(?P<change>\[[^\[\]]+\])? *', expand=False)
    new_start = extract_start.extracted.apply(lambda x: str(x).rstrip())
#     new_end = extract_end.extracted.apply(lambda x: str(x).rstrip())
    each_df['start station'] = new_start
#     each_df['end station'] = new_end
    # Then get the station change
    if extract_start.shape[1] > 1:
        station_change.append(extract_start.dropna(how='any').drop_duplicates())
#     if extract_end.shape[1] > 1:
#         station_change.append(extract_end.dropna(how='any').drop_duplicates())
    i += 1
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [101]:
station_df = pd.read_csv('station.csv')
print station_df.columns
useful_station_df = station_df[[u'lat', u'long', u'name', u'terminalname']]
print useful_station_df.dtypes
print historical_df_list[-1].dtypes

Index([u'id', u'lat', u'long', u'name', u'terminalname'], dtype='object')
lat             float64
long            float64
name             object
terminalname      int64
dtype: object
start date              object
end date                object
start station           object
start station number     int64
dtype: object


In [93]:
combined = []
i = 0
# Find the useful information from the station_df
for each_df in historical_df_list:
    print i
    if "start station number" in each_df.columns:
        # merge on the numbers
        merged = each_df.merge(useful_station_df, left_on='start station number', right_on='terminalname', how='inner')
        combined.append(merged[['start date','end date','start station','terminalname', 'lat', 'long']])
    else:
        # have to merge on the names:
        merged = each_df.merge(useful_station_df, left_on='start station', right_on='name', how='inner')
        combined.append(merged[['start date','end date','start station','terminalname', 'lat', 'long']])
    i += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [94]:
total_df = pd.concat(combined)
print total_df.shape

(11815281, 6)


In [95]:
total_df.to_csv("merged_data.csv", index = False)

In [99]:
sample_df = total_df.sample(frac=0.01) 

In [100]:
sample_df.to_csv("sampled_merged_data.csv", index=False)